# Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

from pyopia.classify import Classify
from pyopia.process import CalculateStats
import pyopia.io
import exampledata
from pyopia.pipeline import Pipeline
import pyopia.instrument.holo as holo

In [3]:
filename = '001-2082.pgm'

In [4]:
# example future pre-processing admin:
# 1) config file config load
# 2) build steps dict from loaded file
# 3) establish non-processing related metadata

datafile_hdf = 'proc/holotest'
model_path = exampledata.get_example_model()
threshold = 0.9

holo_common_settings = {'pixel_size': 4.4, # pixel size in um
                        'wavelength': 658, # laser wavelength in nm
                        'minZ': 22, # minimum reconstruction distance in mm
                        'maxZ': 60, # maximum reconstruction distance in mm
                        'stepZ': 2} #step size in mm

steps = {'common': holo.Common('imbg.pgm', **holo_common_settings),
         'classifier': Classify(model_path=model_path),
         'load': holo.Load(filename),
         'imageprep': holo.Reconstruct(stack_clean=0),
         'statextract': CalculateStats(threshold=threshold),
         'output': pyopia.io.StatsH5(datafile_hdf)}

processing_pipeline = Pipeline(steps)


Load background from disc. 
Build kernel
HoloCommon done 2022-11-06 00:08:40.377991


In [5]:
stats = processing_pipeline.run()

correct background
forward transform
inverse transform
clean stack
summarise stack
segment
clean
2.5% saturation
measure
  81 particles found
WARNING. exportparticles temporarily modified for 2-d images without color!
EXTRACTING 81 IMAGES from 81


In [6]:
stats

,major_axis_length,minor_axis_length,equivalent_diameter,solidity,minr,minc,maxr,maxc,probability_oil,probability_other,probability_bubble,probability_faecal_pellets,probability_copepod,probability_diatom_chain,probability_oily_gas,export name,timestamp,saturation
0,10.490219,7.428310,7.898654,0.731343,7.0,1124.0,17.0,1133.0,0.325525,0.071786,0.483174,0.00343,0.002331,0.005164,0.10859,D20221106T000840.545118-PN0,2022-11-06 00:08:40.545118,2.546354
1,11.783838,8.009105,9.166996,0.767442,45.0,1282.0,57.0,1293.0,0.325525,0.071786,0.483174,0.00343,0.002331,0.005164,0.10859,D20221106T000840.545118-PN1,2022-11-06 00:08:40.545118,2.546354
2,9.290133,5.992048,6.482045,0.733333,70.0,1024.0,78.0,1032.0,0.325525,0.071786,0.483174,0.00343,0.002331,0.005164,0.10859,D20221106T000840.545118-PN2,2022-11-06 00:08:40.545118,2.546354
3,50.017482,3.752744,11.055813,0.494845,80.0,291.0,115.0,316.0,0.325525,0.071786,0.483174,0.00343,0.002331,0.005164,0.10859,D20221106T000840.545118-PN3,2022-11-06 00:08:40.545118,2.546354
4,8.472370,7.527510,6.770275,0.800000,83.0,1081.0,90.0,1089.0,0.325525,0.071786,0.483174,0.00343,0.002331,0.005164,0.10859,D20221106T000840.545118-PN4,2022-11-06 00:08:40.545118,2.546354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,5.414847,4.064513,4.652426,0.944444,1083.0,1104.0,1088.0,1109.0,0.325525,0.071786,0.483174,0.00343,0.002331,0.005164,0.10859,D20221106T000840.545118-PN76,2022-11-06 00:08:40.545118,2.546354
77,8.029648,5.095998,5.527906,0.750000,1084.0,1108.0,1090.0,1116.0,0.325525,0.071786,0.483174,0.00343,0.002331,0.005164,0.10859,D20221106T000840.545118-PN77,2022-11-06 00:08:40.545118,2.546354
78,5.993310,4.003854,4.787307,0.900000,1087.0,1118.0,1093.0,1122.0,0.325525,0.071786,0.483174,0.00343,0.002331,0.005164,0.10859,D20221106T000840.545118-PN78,2022-11-06 00:08:40.545118,2.546354
79,8.535871,3.658991,4.918491,0.703704,1087.0,1131.0,1096.0,1135.0,0.325525,0.071786,0.483174,0.00343,0.002331,0.005164,0.10859,D20221106T000840.545118-PN79,2022-11-06 00:08:40.545118,2.546354


In [7]:

# display metadata in the h5
pyopia.io.show_h5_meta(datafile_hdf + '-STATS.h5')

Modified:
    2022-11-06 00:08:44.172735
Pipeline steps:
    
1) Step: common
   Type: <class 'pyopia.instrument.holo.Common'>
   Vars: {'filename': 'imbg.pgm', 'pixel_size': 4.4, 'wavelength': 658, 'minZ': 22, 'maxZ': 60, 'stepZ': 2}
2) Step: classifier
   Type: <class 'pyopia.classify.Classify'>
   Vars: {'model_path': 'keras_model.h5', 'class_labels': Index(['oil', 'other', 'bubble', 'faecal_pellets', 'copepod', 'diatom_chain',
       'oily_gas'],
      dtype='object'), 'model': <keras.engine.sequential.Sequential object at 0x000002470AA45D90>}
3) Step: load
   Type: <class 'pyopia.instrument.holo.Load'>
   Vars: {'filename': '001-2082.pgm'}
4) Step: imageprep
   Type: <class 'pyopia.instrument.holo.Reconstruct'>
   Vars: {'stack_clean': 0}
5) Step: statextract
   Type: <class 'pyopia.process.CalculateStats'>
   Vars: {'minimum_area': 12, 'threshold': 0.9, 'real_time_stats': False, 'max_coverage': 30, 'max_particles': 5000}
6) Step: output
   Type: <class 'pyopia.io.StatsH5'>
   Var

In [8]:
# load the stats DataFrame from the h5 file
stats = pd.read_hdf(datafile_hdf + '-STATS.h5', 'ParticleStats/stats')
print('stats header: ', stats.columns)
print('Total number of particles: ', len(stats))

stats header:  Index(['major_axis_length', 'minor_axis_length', 'equivalent_diameter',
       'solidity', 'minr', 'minc', 'maxr', 'maxc', 'probability_oil',
       'probability_other', 'probability_bubble', 'probability_faecal_pellets',
       'probability_copepod', 'probability_diatom_chain',
       'probability_oily_gas', 'export name', 'timestamp', 'saturation'],
      dtype='object')
Total number of particles:  212
